In [1]:
import geopandas as gpd
import pandas as pd
import shapely
import haversine
import numpy as np

import os

In [2]:
simplify_dir = 'Stop Snapping/'

In [3]:
def hs(df):
    return haversine.haversine((df['int_lon'], df['int_lat']), (df['stop_lon'], df['stop_lat']), unit=haversine.Unit.METERS)

In [4]:
new_stn = gpd.read_file(simplify_dir + 'new_stations_capture.geojson')

In [5]:
new_stn['name'] = new_stn['name'].str.split(' -', expand = True)[0]

In [6]:
stations = pd.read_csv(simplify_dir + 'stations.csv')

In [7]:
new_stn = new_stn.merge(stations[['name','INT_ID']])

In [8]:
new_stn = new_stn.rename(columns = {'latitude':'int_lat', 'longitude':'int_lon', 'stop_lon_19':'stop_lon', 'stop_lat_19':'stop_lat'})



In [9]:

maj = gpd.read_file(simplify_dir + 'major_capture.geojson')[['INT_ID', 'stop_id']]
maj_min = gpd.read_file(simplify_dir + 'maj_min_capture.geojson')[['INT_ID', 'stop_id']]
min_min = gpd.read_file(simplify_dir + 'min_min_capture.geojson')[['INT_ID', 'stop_id']]
maj_col = gpd.read_file(simplify_dir + 'maj_col_capture.geojson')[['INT_ID', 'stop_id']]
voronoi = gpd.read_file(simplify_dir + 'int_remaining_capture.geojson')[['INT_ID', 'stop_id']]

In [10]:
int_stop = maj.append(maj_min).append(maj_col).append(min_min).append(voronoi).append(new_stn)
int_stop = int_stop.dropna(subset = ['stop_id'])

In [11]:
int_geo = gpd.read_file('/Volumes/Data2/RST/notebook/GIS/intersection-file-wgs84/CENTRELINE_INTERSECTION_WGS84.shp')
int_geo['int_lon'] = int_geo['geometry'].x
int_geo['int_lat'] = int_geo['geometry'].y

int_geo = int_geo[['INT_ID', 'int_lon', 'int_lat']]

In [12]:
stops = pd.read_csv('gtfs_2019-11-14/stops.txt')
stops = stops[['stop_id', 'stop_lon', 'stop_lat']]

In [13]:
int_stop = int_stop.merge(int_geo).merge(stops)[['INT_ID', 'int_lon', 'int_lat', 'stop_id', 'stop_lon', 'stop_lat']]

int_stop_16 = pd.read_csv('/Volumes/Data2/RST/notebook/GTFS/stop_snapping/GIS Simplify/int_stop.csv')[['INT_ID','int_lon', 'int_lat', 'stop_id', 'stop_lon', 'stop_lat']]
int_stop = int_stop.append(int_stop_16)

stn = gpd.read_file(simplify_dir + 'station_capture.geojson').drop_duplicates()
stn = stn.merge(pd.read_csv(simplify_dir + 'stations.csv'))


stn = stn.rename(columns = {'latitude':'int_lat', 'longitude':'int_lon', 'stop_lon_19':'stop_lon', 'stop_lat_19':'stop_lat'})

stn = stn[['INT_ID', 'int_lon', 'int_lat', 'stop_id', 'stop_lon', 'stop_lat']].dropna()

new_stn = gpd.read_file(simplify_dir + 'new_stations_capture.geojson')
new_stn['name'] = new_stn['name'].str.split(' -', expand = True)[0]
stations = pd.read_csv(simplify_dir + 'stations.csv')
new_stn = new_stn.merge(stations[['name','INT_ID']])
new_stn = new_stn.rename(columns = {'latitude':'int_lat', 'longitude':'int_lon', 'stop_lon_19':'stop_lon', 'stop_lat_19':'stop_lat'})
new_stn = new_stn[['INT_ID', 'int_lon', 'int_lat', 'stop_id', 'stop_lon', 'stop_lat']].dropna()




int_stop = int_stop.append(stn).append(new_stn)

int_stop = int_stop.drop_duplicates()
int_stop['dist'] = int_stop.apply(hs, axis = 1)

In [14]:
int_stop = int_stop.sort_values(by = ['INT_ID'])
int_stop = int_stop.drop_duplicates(subset = ['stop_id'], keep = 'first')

In [15]:
int_stop.to_csv('int_stop_19.csv', index = False)

In [16]:
df = pd.read_csv('cleaned_surface/raw_surface_2019-11-28_AM_.csv')

In [17]:
os.listdir('cleaned_surface')[1].split('_')[2]

'2019-11-29'

In [18]:
for file in os.listdir('cleaned_surface'):
    
    period = file.split('_')[3]
    date = file.split('_')[2]

    df = pd.read_csv('cleaned_surface/' + file)
    
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.sort_values(by = ['id', 'timestamp'])

    g = df.groupby('id', as_index=False)
    df['sequence'] = g.cumcount() + 1

    df['hr'] = df['timestamp'].dt.hour
    df['min'] = df['timestamp'].dt.minute
    df['sec'] = df['timestamp'].dt.second
    df_simple = df[['id', 'route', 'stop_id', 'sequence', 'hr', 'min','sec']]

    df_simple['min'] = np.where(df_simple['sec']>29, df_simple['min'] + 1, df_simple['min'])

    df_simple = df_simple.reset_index()

    df_int = df_simple.merge(int_stop[['INT_ID', 'dist', 'stop_id']], how = 'left')

    df_int = df_int.dropna(subset =['INT_ID'
                                      ]).sort_values(by = 'dist', ascending = True).drop_duplicates(subset = [
        'id', 'INT_ID'], keep = 'first').sort_values(by = ['id', 'sequence'])

    g = df_int.groupby('id', as_index=False)
    df_int['stop_sequence_new'] = g.cumcount() + 1

    df_int['hr'] = np.where(df_int['min'] == 60, 
                                        df_int['hr'] + 1, df_int['hr'])
    df_int['min'] = np.where(df_int['min'] == 60, 
                                        0, df_int['min'])
    
    df_int[['id', 'route', 'stop_sequence_new', 'sequence','hr', 'min', 'INT_ID']].to_csv(
    'processed_surface/processed-surface_' + date + '_' + period + '.csv', index = False)
    
    

/var/folders/zw/x46df3ms6r5391lyb_y98z600000gn/T/ipykernel_32058/3802096696.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_simple['min'] = np.where(df_simple['sec']>29, df_simple['min'] + 1, df_simple['min'])
/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
new_ints = int_stop[['INT_ID', 'int_lon', 'int_lat']].drop_duplicates()

new_ints_geo = gpd.GeoDataFrame(new_ints,  geometry=gpd.points_from_xy(
    new_ints.int_lon, new_ints.int_lat)).set_crs(epsg = '4326').to_crs(epsg = '32617')

In [20]:
new_stops = int_stop[['stop_id', 'stop_lon', 'stop_lat']].drop_duplicates()

new_stops_geo = gpd.GeoDataFrame(new_stops,  geometry=gpd.points_from_xy(
    new_stops.stop_lon, new_stops.stop_lat)).set_crs(epsg = '4326').to_crs(epsg = '32617')

In [21]:
new_ints_geo.to_file('new_ints.geojson', driver = 'GeoJSON')

In [22]:
new_stops_geo.to_file('new_stops.geojson', driver = 'GeoJSON')